In [1]:
cd ../deepsalience/

/scratch/rmb456/multif0/deepsalience


In [2]:
import multitask_experiment

Using TensorFlow backend.


In [3]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Reshape, Lambda, Permute
from keras.layers.merge import Concatenate, Multiply
from keras.layers.convolutional import Conv2D
from keras.layers.wrappers import TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [4]:
def get_model():
    input_shape = (None, None, 5)
    y0 = Input(shape=input_shape)

    y1_pitch = Conv2D(
        32, (5, 5), padding='same', activation='relu', name='pitch_layer1')(y0) #32
    y1a_pitch = BatchNormalization()(y1_pitch)
    y2_pitch = Conv2D(
        32, (5, 5), padding='same', activation='relu', name='pitch_layer2')(y1a_pitch) #32
    y2a_pitch = BatchNormalization()(y2_pitch)
    y3_pitch = Conv2D(32, (3, 3), padding='same', activation='relu', name='smoothy2')(y2a_pitch) #32
    y3a_pitch = BatchNormalization()(y3_pitch)
    y4_pitch = Conv2D(8, (70, 3), padding='same', activation='relu', name='distribute')(y3a_pitch) #8
    y4a_pitch = BatchNormalization()(y4_pitch)

    y_multif0 = Conv2D(
        1, (1, 1), padding='same', activation='sigmoid', name='multif0_presqueeze')(y4a_pitch)
    multif0 = Lambda(lambda x: K.squeeze(x, axis=3), name='multif0')(y_multif0)

    y_mask = Multiply(name='mask')([y_multif0, y0])
    y1_timbre = Conv2D(
        512, (2, 3), padding='same', activation='relu', name='timbre_layer1')(y_mask) #512
    y1a_timbre = BatchNormalization()(y1_timbre)

    y_concat = Concatenate(name='timbre_and_pitch')([y_multif0, y1a_timbre])
    ya_concat = BatchNormalization()(y_concat)

    y_mel_feat = Conv2D(
        1, (3, 3), padding='same', activation='relu', name='melody_filters')(ya_concat) #32
    ya_mel_feat = BatchNormalization()(y_mel_feat)
    y_mel_feat2 = Conv2D(
        1, (3, 3), padding='same', activation='relu', name='melody_filters2')(ya_mel_feat)#32
    ya_mel_feat2 = BatchNormalization()(y_mel_feat2)
    y_mel_feat3 = Conv2D(
        1, (240, 1), padding='same', activation='relu', name='melody_filters3')(ya_mel_feat2) # 8
    ya_mel_feat3 = BatchNormalization()(y_mel_feat3)
    y_mel_feat4 = Conv2D(
        1, (7, 7), padding='same', activation='relu', name='melody_filters4')(ya_mel_feat3) # 16
    ya_mel_feat4 = BatchNormalization()(y_mel_feat4)
    y_mel_feat5 = Conv2D(
        1, (7, 7), padding='same', activation='relu', name='melody_filters5')(ya_mel_feat4) #16
    ya_mel_feat5 = BatchNormalization()(y_mel_feat5)

    y_bass_feat = Conv2D(
        1, (3, 3), padding='same', activation='relu', name='bass_filters')(ya_concat) #32
    ya_bass_feat = BatchNormalization()(y_bass_feat)
    y_bass_feat2 = Conv2D(
        1, (3, 3), padding='same', activation='relu', name='bass_filters2')(ya_bass_feat) #32
    ya_bass_feat2 = BatchNormalization()(y_bass_feat2)
    y_bass_feat3 = Conv2D(
        1, (240, 1), padding='same', activation='relu', name='bass_filters3')(ya_bass_feat2) #8
    ya_bass_feat3 = BatchNormalization()(y_bass_feat3)
    y_bass_feat4 = Conv2D(
        1, (7, 7), padding='same', activation='relu', name='bass_filters4')(ya_bass_feat3) #16
    ya_bass_feat4 = BatchNormalization()(y_bass_feat4)
    y_bass_feat5 = Conv2D(
        1, (7, 7), padding='same', activation='relu', name='bass_filters5')(ya_bass_feat4) #16
    ya_bass_feat5 = BatchNormalization()(y_bass_feat5)

    y_vocal_feat = Conv2D(
        1, (3, 3), padding='same', activation='relu', name='vocal_filters')(ya_concat) #32
    ya_vocal_feat = BatchNormalization()(y_vocal_feat)
    y_vocal_feat2 = Conv2D(
        1, (3, 3), padding='same', activation='relu', name='vocal_filters2')(ya_vocal_feat) #32
    ya_vocal_feat2 = BatchNormalization()(y_vocal_feat2)
    y_vocal_feat3 = Conv2D(
        1, (240, 1), padding='same', activation='relu', name='vocal_filters3')(ya_vocal_feat2) #8
    ya_vocal_feat3 = BatchNormalization()(y_vocal_feat3)
    y_vocal_feat4 = Conv2D(
        1, (7, 7), padding='same', activation='relu', name='vocal_filters4')(ya_vocal_feat3) # 16
    ya_vocal_feat4 = BatchNormalization()(y_vocal_feat4)
    y_vocal_feat5 = Conv2D(
        1, (7, 7), padding='same', activation='relu', name='vocal_filters5')(ya_vocal_feat4) #16
    ya_vocal_feat5 = BatchNormalization()(y_vocal_feat5)

    y_melody = Conv2D(
        1, (1, 1), padding='same', activation='sigmoid', name='melody_presqueeze')(ya_mel_feat5)
    melody = Lambda(lambda x: K.squeeze(x, axis=3), name='melody')(y_melody)

    y_bass = Conv2D(
        1, (1, 1), padding='same', activation='sigmoid', name='bass_presqueeze')(ya_bass_feat5)
    bass = Lambda(lambda x: K.squeeze(x, axis=3), name='bass')(y_bass)

    y_vocal = Conv2D(
        1, (1, 1), padding='same', activation='sigmoid', name='vocal_presqueeze')(ya_vocal_feat5)
    vocal = Lambda(lambda x: K.squeeze(x, axis=3), name='vocal')(y_vocal)

    model = Model(inputs=y0, outputs=[multif0, melody, bass, vocal])

    model.summary(line_length=120)

    return model

In [5]:
model = get_model()
output_path = '../experiment_output/trialrun_multitask'
tasks = ['multif0', 'vocal']
data_types = ['XA', 'XD']
loss_weights = {'multif0': 2.0, 'vocal': 1.0} #{'multif0': 2.0, 'melody': 1.0, 'bass': 1.0, 'vocal': 1.0}
sample_weight_mode = {'multif0': None, 'vocal': None}#{'multif0': None, 'melody': None, 'bass': None, 'vocal': None}
task_indices = {'multif0': 0, 'vocal': 1}#{'multif0': 0, 'melody': 1, 'bass': 2, 'vocal': 3}

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   (None, None, None, 5)      0                                                     
________________________________________________________________________________________________________________________
pitch_layer1 (Conv2D)                  (None, None, None, 1)      126                                                   
________________________________________________________________________________________________________________________
batch_normalization_1 (BatchNormalizat (None, None, None, 1)      4                                                     
________________________________________________________________________________________________________________________
pitch_layer2 (Conv2D)           

In [6]:
multitask_experiment.main(
    model, output_path, loss_weights, sample_weight_mode,
    task_indices, data_types=data_types, tasks=tasks, mux_weights=None,
    samples_per_epoch=10, nb_epochs=5, nb_val_samples=10
)

Epoch 1/5
10/10 [==============================] - 5s - loss: 2.2816 - multif0_loss: 0.6959 - vocal_loss: 0.8898 - multif0_mean_squared_error: 0.2496 - multif0_soft_binary_accuracy: 0.3889 - vocal_mean_squared_error: 0.3503 - vocal_soft_binary_accuracy: 0.2458 - val_loss: 2.0774 - val_multif0_loss: 0.6906 - val_vocal_loss: 0.6962 - val_multif0_mean_squared_error: 0.2464 - val_multif0_soft_binary_accuracy: 0.9895 - val_vocal_mean_squared_error: 0.2505 - val_vocal_soft_binary_accuracy: 0.0223
Epoch 2/5
10/10 [==============================] - 0s - loss: 2.2349 - multif0_loss: 0.6896 - vocal_loss: 0.8557 - multif0_mean_squared_error: 0.2470 - multif0_soft_binary_accuracy: 0.4869 - vocal_mean_squared_error: 0.3351 - vocal_soft_binary_accuracy: 0.2182 - val_loss: 2.0671 - val_multif0_loss: 0.6874 - val_vocal_loss: 0.6922 - val_multif0_mean_squared_error: 0.2449 - val_multif0_soft_binary_accuracy: 0.9903 - val_vocal_mean_squared_error: 0.2485 - val_vocal_soft_binary_accuracy: 0.0975
Epoch 3/

/home/rmb456/anaconda2/lib/python2.7/site-packages/mir_eval/multipitch.py:410: UserWarning: Estimate times not equal to reference times. Resampling to common time base.
  warnings.warn("Estimate times not equal to reference times. "
/home/rmb456/anaconda2/lib/python2.7/site-packages/mir_eval/multipitch.py:275: UserWarning: Estimate frequencies are all empty.
  warnings.warn("Estimate frequencies are all empty.")


    > 2 / 27
    > 3 / 27
    > 4 / 27
    > 5 / 27
    > 6 / 27
    > 7 / 27
    > 8 / 27
    > 9 / 27
    > 10 / 27
    > 11 / 27
    > 12 / 27
    > 13 / 27
    > 14 / 27
    > 15 / 27
    > 16 / 27
    > 17 / 27
    > 18 / 27
    > 19 / 27
    > 20 / 27
    > 21 / 27
    > 22 / 27
    > 23 / 27
    > 24 / 27
    > 25 / 27
    > 26 / 27
    > 27 / 27
Best Threshold is 0.1
Best validation accuracy is 0.0
Validation accuracy at 0.5 is 0.0
    > Scoring on bach10...
    > 1 / 10
    > 2 / 10
    > 3 / 10
    > 4 / 10
    > 5 / 10
    > 6 / 10
    > 7 / 10
    > 8 / 10
    > 9 / 10
    > 10 / 10
       Precision  Recall  Accuracy  Substitution Error  Miss Error  \
count       10.0    10.0      10.0           10.000000   10.000000   
mean         0.0     0.0       0.0            0.035384    0.964616   
std          0.0     0.0       0.0            0.008367    0.008367   
min          0.0     0.0       0.0            0.020123    0.951424   
25%          0.0     0.0       0.0            0.

IndexError: list index out of range